# Signs of Elementary Differentials in Expansions

With this notebook, you can experiment with the expansions of the energy change during one step of Runge-Kutta methods as described in the paper.

Run the cell below to activate the project environment containing the necessary packages.

In [ ]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

# Setup

Run this cell once to setup some basic functions.

In [ ]:
using RootedTrees
using LinearAlgebra
using StaticArrays, DataStructures

function doit(A, b, c, max_order_Δt)
    println("Residual Order Conditions:")
    for order in 1:max_order_Δt+1
        println("Order ", order, ":")
        for t in RootedTreeIterator(order)
            println("  ", residual_order_condition(t, A, b, c))
        end
    end
    println()
    
    M = b*b' - Diagonal(b)*A - A'*Diagonal(b)

    #terms = OrderedDict{NTuple{2,RootedTree{Int,Vector{Int}}}, eltype(A)}()
    terms = DefaultOrderedDict(0)
    for order in 0:max_order_Δt
        for order1 in 1:order+1, t1 in RootedTreeIterator(order1)
            order2 = order+2 - order1
            for t2 in RootedTreeIterator(order2)
                if order1 > order2 || (order1 == order2 && t1 < t2)
                    trees = (copy(t2),copy(t1))
                else
                    trees = (copy(t1),copy(t2))
                end
                
                ΦᵢD = derivative_weight(t1, A, b, c) / σ(t1)
                ΦⱼD = derivative_weight(t2, A, b, c) / σ(t2)
                terms[trees] += ΦᵢD' * M * ΦⱼD
            end
        end
    end
    
    terms
end

function printit(terms)
    order_Δt = -1
    for trees in keys(terms)
        t1, t2 = trees
        new_order_Δt = order(t1)+order(t2)-2
        if new_order_Δt != order_Δt
            order_Δt = new_order_Δt
            println("Δt^", order_Δt, ":")
        end
        println("  ", trees)
        println("  ", terms[trees])
        println()
    end
end


# Expansions for Classical Algebraically Stable Schemes

The residuals of the order conditions are printed at first. This can be used to verify the order of a given method. Afterwards, the terms appearing in the expansion are printed. The rooted trees appearing in the braces have to be substituted by the scalar product of the corresponding elementary differentials. The notation used for the rooted trees is described in the Julia package [RootedTrees.jl](https://github.com/JuliaDiffEq/RootedTrees.jl).

In [ ]:
# Lobatto IIIC, Order 2
A = @SArray [1//2 -1//2; 1//2 1//2]
b = @SArray [1//2, 1//2]
c = A * SVector(1, 1)

# Radau IIA, Order 3
A = @SArray [5//12 -1//12; 3//4 1//4]
b = @SArray [3//4, 1//4]
c = A * SVector(1, 1)

# compute the terms of the expansion up to a maximal order Δt^3
terms = doit(A, b, c, 3)
printit(terms)

## Expansions for Some Special Schemes

These schemes are reducible and equivalent to the algebraically (and $B$) stable implicit  midpoint method.

In [ ]:
#= Hairer, Nørsett, Wanner II, Table IV.12.2
# -> implicit midpoint, all terms are zero
A = @SArray [1//2 0; 1//4 1//4]
b = @SArray [1//2, 1//2]
c = A * SVector(1, 1)=#

# Crouzeix (1979), p. 80
# -> implicit midpoint, all terms are zero
A = @SArray [1//2 0; 0 1//2]
b = @SArray [2, -1]
c = A * SVector(1, 1)

terms = doit(A, b, c, 7)
printit(terms)

## SSPRK(3,3) and a First Order Stable Method

This method is not energy stable for semibounded problems.

In [ ]:
# SSPRK33
A = @SArray [0 0 0; 1 0 0; 1//4 1//4 0]
b = @SArray [1//6, 1//6, 2//3]
c = A * SVector(1, 1, 1)

# First Order Stable Method of H. Ranocha, 
# On Strong Stability of Explicit Runge-Kutta Methods for Nonlinear Semibounded Operators,
# https://arxiv.org/abs/1811.11601
#A = @SArray [0 0; 3//2 0]
#b = @SArray [1//2, 1//2]
#c = A * SVector(1, 1)

# the classical matrix used to define algebraically stable schemes
M = b*b' - Diagonal(b)*A - A'*Diagonal(b)
display(M)
display(eigvals(Float64.(M)))

f_and_higher_differentials = b - b .* c - A' * b
display(f_and_higher_differentials)

for k in 1:9
    println("k = ", k)
    #println("  ", (b' * c.^k)^2)
    #println("  ", -2 * (c.^k .* b)' * A * c.^k)
    println("  ", ((b' * c.^k)^2 - 2 * (c.^k .* b)' * A * c.^k) / factorial(k)^2)
end

terms = doit(A, b, c, 6)
printit(terms)

## Second Order Stable Method

This explicit second order method is conditionally energy stable for semibounded autonomous problems.

In [ ]:
A = @SArray [0 0 0 0; 1 0 0 0; 1 -1 0 0; -1 1 1 0//1]
b = @SArray [1//4, 1//4, 1//4, 1//4]
c = A * SVector(1, 1, 1, 1)

# the classical matrix used to define algebraically stable schemes
M = b*b' - Diagonal(b)*A - A'*Diagonal(b)
display(M)
display(eigvals(Float64.(M)))

f_and_higher_differentials = b - b .* c - A' * b
display(f_and_higher_differentials)

for k in 1:9
    println("k = ", k)
    #println("  ", (b' * c.^k)^2)
    #println("  ", -2 * (c.^k .* b)' * A * c.^k)
    println("  ", ((b' * c.^k)^2 - 2 * (c.^k .* b)' * A * c.^k) / factorial(k)^2)
end

terms = doit(A, b, c, 6)
printit(terms)

# Third Order Stable Method

This explicit third order method is conditionally energy stable for semibounded autonomous problems.

In [ ]:
A = @SArray [0 0 0 0 0; 1//2 0 0 0 0; 1 0 0 0 0; 1 0 -1 0 0; -3 2 1 1 0]
b = @SArray [0, 2//3, 0, 1//6, 1//6]
c = A * (zero(b) .+ 1)

# the classical matrix used to define algebraically stable schemes
M = b*b' - Diagonal(b)*A - A'*Diagonal(b)
display(M)
display(eigvals(Float64.(M)))

f_and_higher_differentials = b - b .* c - A' * b
display(f_and_higher_differentials)

for k in 1:9
    println("k = ", k)
    #println("  ", (b' * c.^k)^2)
    #println("  ", -2 * (c.^k .* b)' * A * c.^k)
    println("  ", ((b' * c.^k)^2 - 2 * (c.^k .* b)' * A * c.^k) / factorial(k)^2)
end

terms = doit(A, b, c, 6)
printit(terms)

### A Computational Verification of a Calculation the Paper

In [ ]:
import SymPy
k = SymPy.symbols("k", integer=true, positive=true)

expr = -11//36 + 4//9 * 2^(-k) * (2^(-k) - 1)
SymPy.expand((c.^k)' * M * (c.^k) - expr) 

# Some Analytical Expressions

You can also use the provided utilities to generate the expansions and order conditions for symbolic inputs.

In [ ]:
using SymEngine

# this hack needs to be added to get the inner products correct
Base.conj(b::Basic) = b

A = Basic.(@SArray [0 0 0; "a21" 0 0; "a31" "a32" 0])
b = Basic.(@SVector ["b1", "b2", "b3"])
c = A * SVector(1, 1, 1)

terms = doit(A, b, c, 3)
printit(terms)